# ACID upload of data objects

Here we explore the ACID behavior of LaminDB's upload API.

In [ ]:
from lndb import init, load, delete, settings
from lndb.dev import setup_local_test_postgres, UPath

pgurl = setup_local_test_postgres()
init(name="acidtests", storage="acidtests", db=pgurl)

In [ ]:
import lamindb as ln
import lamindb.schema as lns

ln.nb.header()

## Ingestion failure due to failed upload to storage

Let's try to ingest a data object to a storage location without permission.

In [ ]:
import pytest

# Create data object
adata = ln.dev.datasets.anndata_mouse_sc_lymph_node()
dobject = ln.DObject(adata, name="Mouse Lymph Node scRNA-seq")

# Update storage path with corrupt path
settings.instance.storage._root = UPath("s3://nf-core-awsmegatests")

# Ingest data object
with pytest.raises(RuntimeError) as e:
    error = e
    added_dobject = ln.add(dobject)
print(error.exconly())

Let's check that no metadata records were added to the database.

In [ ]:
dobjects = ln.select(ln.DObject).all()
assert len(dobjects) == 0

## Ingestion failure due to failed database transaction

Let's try to add the same `Project` record twice, violating the primary key unique constraint.

In [ ]:
added_project = ln.add(lns.Project(name="test-project"))
with pytest.raises(RuntimeError) as e:
    error = e
    ln.add(lns.Project, id=added_project.id, name="conflict-project")
print(error.exconly())

## Ingestion failure during list-based ingestion

If a list of data objects is passed to `ln.add()` and the upload of one of these data objects fails, the successful uploads are maintained and a `RuntimeError` is raised, listing the successfully uploaded data objects up until that point.

## Ingestion failure unrelated to upload to storage or DB transaction

Let's now restore the storage location.

In [ ]:
load("acidtests")

Errors that are not related to database connection or file upload are raised with their original exception.

No entries are committed to the database or uploaded to storage.

In [ ]:
from sqlalchemy.orm.exc import UnmappedInstanceError

filepath = ln.dev.datasets.file_jpg_paradisi05()
dobject = ln.DObject(filepath)
dobjects = [dobject, "this is not a data object"]
with pytest.raises(UnmappedInstanceError) as e:
    exception = e
    ln.add(dobjects)
print(exception.exconly())

In [ ]:
dobjects = ln.select(ln.DObject).all()
assert len(dobjects) == 0

In [ ]:
!docker stop pgtest && docker rm pgtest

In [ ]:
delete("acidtests")